# EarlyStruct – Design Explorer (Floors)
Carbon-first feasibility & comparison tool.

**How to use**
1) Set **Data dir** to the folder containing your CSVs.
2) (Optional) Provide a **Control file** path. If present, it takes precedence for project details and SPANS unless it contains `USE_CSV = Y`.
3) (Optional) Enter **Spans** like `9, 10.5` (meters) or `28ft, 32ft`. Notebook spans override control-file SPANS.
4) Click **Run Evaluation**. If no spans are provided (and no grid_options.csv / ideal spacing), the tool runs a 1-ft sweep (18–45 ft).


In [21]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
pkg_root = os.path.abspath(os.path.join('..'))
if pkg_root not in sys.path:
    sys.path.insert(0, pkg_root)
from earlystruct.cli import evaluate
print('earlystruct package ready')


earlystruct package ready


In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Output
import importlib
import matplotlib.pyplot as plt

from earlystruct.cli import evaluate
from earlystruct.core import reporting

out = out if 'out' in globals() else Output()

SHOW_COLS = [
    "span_m","system_id","system_name","category","type","manufacturer",
    "depth_m","carbon_per_m2","cost_per_m2","edge_canti_x_m","edge_canti_y_m",
    "reason","on_pareto"
]

def _annotate_pareto(df):
    """Add an on_pareto boolean based on feasible rows with both metrics."""
    df2 = df.copy()
    df2["on_pareto"] = False
    feas = df2[(df2["feasible"] == True) &
               df2["carbon_per_m2"].notna() &
               df2["cost_per_m2"].notna()].copy()
    if not feas.empty:
        mask = reporting.pareto_mask(feas, "carbon_per_m2", "cost_per_m2")
        df2.loc[feas.index, "on_pareto"] = mask.values
    return df2

def on_run(_):
    with out:
        clear_output(wait=True)
        status.value = "<b>Running…</b>"
        try:
            cf = control_file.value.strip() or None
            df, ranked, pareto_df, _ = evaluate(data_dir.value, spans.value or None, None, cf)

            # charts + summary blocks
            render_results(df)

            # annotate pareto for the tabular view
            df_ann = _annotate_pareto(df)
            feas = df_ann[df_ann["feasible"]].copy()

            status.value = f"<b>Done.</b> All: {len(df_ann)} | Feasible: {len(feas)}"
            cols = [c for c in SHOW_COLS if c in feas.columns]
            display(feas[cols].sort_values(["carbon_per_m2","cost_per_m2"], na_position="last").reset_index(drop=True))

        except Exception as e:
            status.value = f'<span style="color:red">Error: {e}</span>'

def render_results(df):
    """Show best-per-typology and best-per-type tables + two charts."""
    clear_output(wait=True)

    # ---- Tables ----
    print("Lowest-carbon option per TYPOLOGY (feasible only):")
    best_carb_typ = reporting.best_per_typology(df, "carbon_per_m2")
    display(best_carb_typ if not best_carb_typ.empty else "— none —")

    print("\nLowest-cost option per TYPOLOGY (feasible only):")
    best_cost_typ = reporting.best_per_typology(df, "cost_per_m2")
    display(best_cost_typ if not best_cost_typ.empty else "— none —")

    print("\nLowest-carbon option per TYPE (feasible only):")
    best_carb_type = reporting.best_per_type(df, "carbon_per_m2")
    display(best_carb_type if not best_carb_type.empty else "— none —")

    print("\nLowest-cost option per TYPE (feasible only):")
    best_cost_type = reporting.best_per_group(df, "cost_per_m2", "type")
    display(best_cost_type if not best_cost_type.empty else "— none —")

    # ---- Charts ----
    reporting.plot_pareto(df, title="Carbon vs Cost — Pareto (feasible)")
    reporting.plot_best_typology_carbon(df, title="Lowest-carbon option by typology")
    reporting.plot_best_type_carbon(df, title="Lowest-carbon option by type")


## Pareto Chart (Carbon vs Cost)
Plot uses the latest run's feasible set.

In [23]:
from IPython.display import display, clear_output
import importlib

# hot-reload in case you tweak files
import earlystruct.core.reporting as reporting
importlib.reload(reporting)

def render_summary(df):
    clear_output(wait=True)

    # --- one table: best (lowest carbon) per typology ---
    best_carb = reporting.best_per_typology(df, "carbon_per_m2")
    print("Lowest-carbon option per typology (feasible only):")
    if not best_carb.empty:
        display(best_carb)
    else:
        print("  (none)")

    # --- one table: best (lowest cost) per typology ---
    best_cost = reporting.best_per_typology(df, "cost_per_m2")
    print("\nLowest-cost option per typology (feasible only):")
    if not best_cost.empty:
        display(best_cost)
    else:
        print("  (none)")

    # --- chart 1: Pareto scatter (carbon vs cost) ---
    reporting.plot_pareto(df, title="Carbon vs Cost — Feasible candidates (Pareto)")

    # --- chart 2: “like the screenshot” horizontal bar of best per typology ---
    reporting.plot_best_typology_carbon(df, title="Lowest-carbon option by typology")


In [24]:
from ipywidgets import Text, Button, VBox, HTML
data_dir = Text(value='/Users/benjaminsalop/Desktop/Oxford/Research/edca/csvs', description='Data dir:', layout={'width':'70%'})
control_file = Text(value='/Users/benjaminsalop/Desktop/Oxford/Research/edca/control_files/control_file.txt', description='Control:', layout={'width':'70%'})
spans = Text(value='6', description='Spans:')
run_btn = Button(description='Run Evaluation', button_style='primary')
status = HTML(value='')
VBox([data_dir, control_file, spans, run_btn, status])

In [25]:
run_btn.on_click(on_run)
out

Output()